In [5]:
import re
from typing import Tuple

In [6]:
doc_read_pattern = re.compile("<doc><docno>([^<]*)</docno>([^<]*)</doc>")


def read_doc(text: str) -> Tuple[str, str]:
    matcher = doc_read_pattern.search(text)
    if matcher == None:
        return None, None
    return matcher.group(1), matcher.group(2)


In [7]:
with open("doc.txt", "r") as f:
    lines = f.readlines()


In [27]:
class IndexObject:
    def __init__(self) -> None:
        self.df = 0
        self.tf = dict()

In [28]:
index = dict()

# Building index

for i in range(len(lines)):
    line = lines[i]
    docno, doctext = read_doc(line)
    if docno == None:
        continue

    words = re.findall('\w+', doctext)
    for w in words:
        word = w.lower()

        if word in index:
            wl = index[word]
        else:
            wl = IndexObject()
            index[word] = wl
        
        if docno not in wl.tf:
            wl.df += 1
            wl.tf[docno] = 1
        else:
            wl.tf[docno] += 1
        


In [29]:
for word in sorted(index):
    io = index[word]
    print("{0}=df({1})".format(io.df, word))
    for doc in sorted(io.tf):
        print("\t{0} {1}".format(io.tf[doc], doc))


1=df(arabia)
	1 D4
1=df(casablanca)
	1 D1
1=df(citezen)
	1 D0
1=df(godfather)
	2 D2
1=df(gone)
	1 D3
1=df(graduate)
	1 D6
1=df(in)
	1 D9
1=df(kane)
	1 D0
1=df(lawrence)
	1 D4
1=df(list)
	1 D8
2=df(of)
	1 D4
	2 D5
1=df(on)
	1 D7
1=df(oz)
	2 D5
1=df(rain)
	1 D9
1=df(s)
	1 D8
1=df(schindler)
	1 D8
1=df(singin)
	1 D9
6=df(the)
	2 D2
	1 D3
	2 D5
	1 D6
	1 D7
	1 D9
1=df(waterfront)
	1 D7
1=df(wind)
	1 D3
1=df(with)
	1 D3
1=df(wizard)
	2 D5
